In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from autodm.llm import get_llm
from autodm.roll import Dice, DiceType
from autodm.character import Character

In [3]:
character = Character.generate(level=0)

In [23]:
from llama_index.core.program import LLMTextCompletionProgram
from pydantic import BaseModel, Field
from typing import Optional, List, Any

In [5]:
LOCATION_TYPES = [
    "region",
    "city",
    "road",
    "dungeon",
    "wilderness",
    "building",
    "room",
]

In [48]:
class Location(BaseModel):
    type: str = Field(..., title="The type of the location", description=f"The type of the location, one of {', '.join(LOCATION_TYPES)}")
    name: str = Field(..., title="The name of the location", description="The name of the location")
    description: str = Field(..., title="The description of the location", description="A description of the location.")
    parent: Optional[Location] = Field(None, title="The parent location", description="The name of the parent location of this location.")
    children: Optional[List[Location]] = Field([], title="The children locations", description="The names of the children locations of this location.")

    def __str__(self):
        return self.name
    
    def __repr__(self):
        return f"Location(name={self.name}, type={self.type}, description={self.description}, parent={self.parent}, children={self.children})"
    
    @classmethod
    def generate(self, **kwargs):
        if 'parent' not in kwargs:
            kwargs['type'] = 'region'
        kwgs = "\n".join([f"{k}: {v}" for k, v in kwargs.items()])
        prompt = f"""Please create a location for a D&D campaign. \
If a parent is passed, please make sure the properties align with the parent location. \
Do not create a child location that is larger than the parent. For example, a building cannot be the child of a room, unless there are alternate dimensions involved. \
The location should have the following properties: 
    {kwgs}"""
        
        program = LLMTextCompletionProgram.from_defaults(prompt_template_str=prompt, output_cls=Location, llm=get_llm())
        return program()
    
    def generate_child(self):
        child = Location.generate(parent=self)
        self.children.append(child)
        

In [36]:
l = Location.generate(type='region')

In [47]:
l.children[0]

Location(name=The Great Forest, type=region, description=A vast and ancient forest covering most of the eastern part of the realm, home to numerous creatures and magical beings., parent=None, children=[Location(name=Elvendale, type=city, description=An elven city nestled deep within the heart of the Great Forest, known for its beautiful architecture and advanced magic., parent=None, children=[]), Location(name=The Darkwoods, type=wilderness, description=A mysterious and dangerous area located in the northern part of the Great Forest, plagued by dark creatures and curses., parent=None, children=[])])

In [39]:
c = l.generate_child()

In [43]:
c.children[0]

Location(name=Elvendale, type=city, description=An elven city nestled deep within the heart of the Great Forest, known for its beautiful architecture and advanced magic., parent=None, children=[])